In [ ]:
### *Data Analysis*
##  Regression analysis on the aggregate 
---
*conducted in April 2022*

In [64]:
# Import necessary libraries
import nltk, re, pprint
import json
from nltk import word_tokenize
from nltk import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
import nltk.data
import os.path 
import glob
import pandas as pd
import os
import re
from nltk.corpus import PlaintextCorpusReader 
from nltk.app import concordance
from nltk.corpus import BracketParseCorpusReader
import numpy as np
import contractions
import statsmodels.formula.api as smf
import altair as alt
import tmtoolkit
import spacy as spacy
import logging, warnings
from tmtoolkit.corpus import Corpus
import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess
nltk.download('omw-1.4')
import pickle
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import pyLDAvis
import pyLDAvis.gensim_models
import docx


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/charlottekaiser/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [110]:
# Load information from topic modelling
topicmodel_df = pd.read_csv(r'/Users/charlottekaiser/Documents/uni/Hertie/master_thesis/00_data/50_analysis/LDA_mallet_aggregate.csv')

In [111]:
# Condense dataset
topicmodel_df
data= topicmodel_df.drop(['Unnamed: 0', 'Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text'], axis=1)

In [112]:
# Add topic definition as defined qualitatively by the author 
topic = ['Democratic freedom', 'Citizens online rights', 'AI education', 'AI education', 'Citizens online rights', 'AI education', 'Citizens online rights', 
'Global market', 'Residual Topic', 'Residual Topic', 'Residual Topic', 'AI education', 'National defense', 'AI education', 
'Residual Topic', 'AI education', 'National defense', 'National defense', 'Domestic Economy']
data["topic"] = topic 

In [113]:
# Drop those observations where the residual topic is most prevalent
data.drop([8, 9, 10, 14]).reset_index()

,index,EU,debate_title,10_most_relevant,topic
0,0,1,EU02_Democratic scrutiny of social media and t...,"0.030*""freedom"" + 0.027*""platform"" + 0.023*""de...",Democratic freedom
1,1,1,EU03_European strategy for data - Commission e...,"0.024*""european"" + 0.017*""datum"" + 0.013*""ensu...",Citizens online rights
2,2,1,EU11_Digital Europe programme.txt,"0.025*""woman"" + 0.022*""technology"" + 0.018*""ed...",AI education
3,3,1,"EU13_Artificial intelligence in education, cul...","0.025*""woman"" + 0.022*""technology"" + 0.018*""ed...",AI education
4,4,1,EU14_Digital future of Europe- digital single ...,"0.024*""european"" + 0.017*""datum"" + 0.013*""ensu...",Citizens online rights
5,5,1,"EU15_ Promoting gender equality in science, te...","0.025*""woman"" + 0.022*""technology"" + 0.018*""ed...",AI education
6,6,1,EU18_Artificial intelligence in criminal law a...,"0.024*""european"" + 0.017*""datum"" + 0.013*""ensu...",Citizens online rights
7,7,1,EU21_The outcome of the EU-US Trade and Techno...,"0.028*""important"" + 0.021*""work"" + 0.018*""trad...",Global market
8,11,0,US09_MSI STEM ACHIEVEMENT ACT.txt,"0.025*""woman"" + 0.022*""technology"" + 0.018*""ed...",AI education
9,12,0,US10_National Defense Authorization Act.txt,"0.045*""military"" + 0.030*""defense"" + 0.012*""me...",National defense


In [114]:
# Create dummies for catgeorical topic variable
cat =['topic']
for var in cat:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(data[var], prefix=var)
    data1=data.join(cat_list)
    data=data1

cat_vars=['topic']
data_vars=data.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]

data=data[to_keep]
data.columns.values

array(['EU', 'debate_title', '10_most_relevant', 'topic_AI education',
       'topic_Citizens online rights', 'topic_Democratic freedom',
       'topic_Domestic Economy', 'topic_Global market',
       'topic_National defense', 'topic_Residual Topic'], dtype=object)

In [115]:
data

,EU,debate_title,10_most_relevant,topic_AI education,topic_Citizens online rights,topic_Democratic freedom,topic_Domestic Economy,topic_Global market,topic_National defense,topic_Residual Topic
0,1,EU02_Democratic scrutiny of social media and t...,"0.030*""freedom"" + 0.027*""platform"" + 0.023*""de...",0,0,1,0,0,0,0
1,1,EU03_European strategy for data - Commission e...,"0.024*""european"" + 0.017*""datum"" + 0.013*""ensu...",0,1,0,0,0,0,0
2,1,EU11_Digital Europe programme.txt,"0.025*""woman"" + 0.022*""technology"" + 0.018*""ed...",1,0,0,0,0,0,0
3,1,"EU13_Artificial intelligence in education, cul...","0.025*""woman"" + 0.022*""technology"" + 0.018*""ed...",1,0,0,0,0,0,0
4,1,EU14_Digital future of Europe- digital single ...,"0.024*""european"" + 0.017*""datum"" + 0.013*""ensu...",0,1,0,0,0,0,0
5,1,"EU15_ Promoting gender equality in science, te...","0.025*""woman"" + 0.022*""technology"" + 0.018*""ed...",1,0,0,0,0,0,0
6,1,EU18_Artificial intelligence in criminal law a...,"0.024*""european"" + 0.017*""datum"" + 0.013*""ensu...",0,1,0,0,0,0,0
7,1,EU21_The outcome of the EU-US Trade and Techno...,"0.028*""important"" + 0.021*""work"" + 0.018*""trad...",0,0,0,0,1,0,0
8,0,US02_CONSUMER SAFETY TECHNOLOGY ACT.txt,"0.036*""support"" + 0.035*""time"" + 0.019*""includ...",0,0,0,0,0,0,1
9,0,US04_FEDERAL CAREER OPPORTUNITIES IN COMPUTER ...,"0.036*""support"" + 0.035*""time"" + 0.019*""includ...",0,0,0,0,0,0,1


In [117]:
X = data.loc[:, data.columns == 'EU']
y = data.loc[:, data.columns == 'topic_Citizens online rights']

In [118]:
import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 34.835251
         Iterations 4
                               Results: Logit
Model:              Logit                        Pseudo R-squared: inf      
Dependent Variable: topic_Citizens online rights AIC:              1325.7396
Date:               2022-04-11 13:05             BIC:              1326.6840
No. Observations:   19                           Log-Likelihood:   -661.87  
Df Model:           0                            LL-Null:          0.0000   
Df Residuals:       18                           LLR p-value:      nan      
Converged:          1.0000                       Scale:            1.0000   
No. Iterations:     4.0000                                                  
---------------------------------------------------------------------------------
           Coef.       Std.Err.         z         P>|z|        [0.025      0.975]
-----------------------------------------------------------------------

/Users/charlottekaiser/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/Users/charlottekaiser/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/Users/charlottekaiser/miniforge3/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:3511: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.llf/self.llnull
